# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
csv_path = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(csv_path)
#cities_df = cities_df.reset_index(drop=True)
#mean_df = mean_df.reset_index(drop=True)
print(cities_df.dtypes)
cities_df.head()


Unnamed: 0           int64
city name           object
country             object
date                 int64
latitude           float64
longitude          float64
max temperature    float64
humidity             int64
cloudiness           int64
wind speed         float64
dtype: object


,Unnamed: 0,city name,country,date,latitude,longitude,max temperature,humidity,cloudiness,wind speed
0,0,Takoradi,GH,1578261797,4.89,-1.75,79.75,58,5,6.42
1,1,New Norfolk,AU,1578261797,-42.78,147.06,62.01,58,20,3.36
2,2,Monte Alegre,BR,1578261797,-2.00,-54.07,85.89,62,80,2.55
3,3,Airai,TL,1578261797,-8.93,125.41,69.42,84,100,5.26
4,4,Seguin,US,1578261797,29.57,-97.96,75.99,22,1,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["latitude", "longitude"]]
#text ='City: '+ cities_df["city name"]
#countries = cities_df["country"].astype(str)
#cities = cities_df["city name"].astype(str)

#text ='<b>City:</b><br> '+ cities + '<br><b>Country:</b><br> ' + countries




#text=text1+text2

humidity = cities_df["humidity"]

# Plot Heatmap
#fig = gmaps.figure()
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layers
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
#city_label_layer = gmaps.marker_layer(locations, info_box_content=text)


# Add layers
#fig.add_layer(city_label_layer)
fig.add_layer(heat_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:

#* A max temperature lower than 80 degrees but higher than 70.

#* Wind speed less than 10 mph.

#* Zero cloudiness.

#* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
cities_df=cities_df.loc[cities_df["max temperature"]>75, :]
cities_df=cities_df.loc[cities_df["max temperature"]<85, :]
cities_df=cities_df.loc[cities_df["cloudiness"]<5, :]
cities_df=cities_df.loc[cities_df["wind speed"]<10, :]
cities_df=cities_df.loc[cities_df["humidity"]<70, :]


#cities_df = pd.DataFrame("city name", "country", "date", "latitude", "longitude", "max temperature", "humidity", "cloudiness", "wind speed" )

cities_df


,Unnamed: 0,city name,country,date,latitude,longitude,max temperature,humidity,cloudiness,wind speed
31,31,Georgetown,GY,1578261531,6.80,-58.16,84.20,61,0,6.93
74,74,Atuona,PF,1578261807,-9.80,-139.03,80.56,67,0,4.74
89,89,Baindur,IN,1578261809,13.87,74.63,75.70,64,0,4.00
228,228,Yeppoon,AU,1578261823,-23.13,150.74,80.60,65,2,6.93
281,281,Cape Coast,GH,1578261828,5.11,-1.24,82.00,54,0,9.86
438,438,Kavaratti,IN,1578261832,10.57,72.64,79.97,63,0,3.85
528,528,Trat,TH,1578261848,12.25,102.51,78.84,69,2,3.47
545,545,Kerouane,GN,1578261849,12.23,-12.34,75.20,29,0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
cities_df["Hotel Name"]=""
hotel_df=cities_df
hotel_df

,Unnamed: 0,city name,country,date,latitude,longitude,max temperature,humidity,cloudiness,wind speed,Hotel Name
31,31,Georgetown,GY,1578261531,6.80,-58.16,84.20,61,0,6.93,
74,74,Atuona,PF,1578261807,-9.80,-139.03,80.56,67,0,4.74,
89,89,Baindur,IN,1578261809,13.87,74.63,75.70,64,0,4.00,
228,228,Yeppoon,AU,1578261823,-23.13,150.74,80.60,65,2,6.93,
281,281,Cape Coast,GH,1578261828,5.11,-1.24,82.00,54,0,9.86,
438,438,Kavaratti,IN,1578261832,10.57,72.64,79.97,63,0,3.85,
528,528,Trat,TH,1578261848,12.25,102.51,78.84,69,2,3.47,
545,545,Kerouane,GN,1578261849,12.23,-12.34,75.20,29,0,4.70,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius":5000,
    "types":"hotel",
    "keyword": "Lodging",
    "key":g_key
}

# Use the lat/lng we recovered to identify airports
for index,row in hotel_df.iterrows():
    #get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    params["location"] = f"{lat},{lng}"
    
    
    # assemble url and make API request
    
    response = requests.get(base_url, params=params).json()
    
    
    if (response["status"]=="OK"):
        hotel_df.loc[index, "Hotel Name"]=response["results"][0]["name"]
    else:
        print("Zero Rezults. Hotel Not Found.")
        hotel_df.loc[index, "Hotel Name"]="NaN"

        
hotel_df = hotel_df.dropna(axis=0, how='any')

hotel_df
    
    

Zero Rezults. Hotel Not Found.


,Unnamed: 0,city name,country,date,latitude,longitude,max temperature,humidity,cloudiness,wind speed,Hotel Name
31,31,Georgetown,GY,1578261531,6.80,-58.16,84.20,61,0,6.93,Herdmanston Lodge
74,74,Atuona,PF,1578261807,-9.80,-139.03,80.56,67,0,4.74,Simplicité Marquises
89,89,Baindur,IN,1578261809,13.87,74.63,75.70,64,0,4.00,Sai Vishram Beach Resort
281,281,Cape Coast,GH,1578261828,5.11,-1.24,82.00,54,0,9.86,Oasis Beach Resort
438,438,Kavaratti,IN,1578261832,10.57,72.64,79.97,63,0,3.85,Bismillah Hotel And Lodging
528,528,Trat,TH,1578261848,12.25,102.51,78.84,69,2,3.47,"Basar Guest House,Trat,Thailand"
545,545,Kerouane,GN,1578261849,12.23,-12.34,75.20,29,0,4.70,NaN


In [22]:
#hotel_df=hotel_df.drop(hotel_df.index[3] )
#hotel_df


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [24]:
#Configure gmaps.
#gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = hotel_df[["latitude", "longitude"]]



info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]




humidity = hotel_df["humidity"]

# Plot Heatmap

#fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layers
#heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
city_label_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Add layers
fig.add_layer(city_label_layer)
#fig.add_layer(heat_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))